# EM for Binary Data

In [89]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, linewidth=100)

In [90]:
dataset = np.loadtxt('binarydigits.txt')
N, D = dataset.shape

N, D

(100, 64)

We have $N = 100$ images, each with $D = 64$ pixels laid out as a vector, i.e., $\mathbf{x}^{\left(n\right)} \in \mathbb{R}^D,\ \forall n\in \left\{1,\ldots,N\right\}$. 

## 3 (d)

In [91]:
class EMForMultivariateBernoulli:

    def log_likelihood(self, X, parameters, pi):

        odds = np.divide(np.minimum(parameters, 1-1e-10), 1-np.minimum(parameters, 1-1e-10))
        marginals = list()
        for x in X:
            likelihoods = np.prod(np.multiply(1-parameters, np.power(odds, x)), axis=1)
            joints = np.multiply(pi, likelihoods)
            marginal = np.sum(joints)
            marginals.append(marginal)
        marginals = np.array(marginals)
        log_likelihood = np.sum(np.log(marginals))

        return log_likelihood

    def expectation(self, X, parameters, priors):

        odds = np.divide(np.minimum(parameters, 1-1e-10), 1-np.minimum(parameters, 1-1e-10))
        responsibilities = list()
        for x in X:
            likelihoods = np.prod(np.multiply(1-parameters, np.power(odds, x)), axis=1)
            joints = np.multiply(priors, likelihoods)
            marginal = np.sum(joints)
            posteriors = joints / marginal
            responsibilities.append(posteriors)
        responsibilities = np.array(responsibilities)
        
        return responsibilities

    def maximisation(self, X, responsibilities):

        P = np.divide(responsibilities.T @ X, np.expand_dims(np.sum(responsibilities, axis=0), axis=1))
        pi = np.mean(responsibilities, axis=0)

        return P, pi

    def __call__(self, K, X, n_iterations, epsilon):

        P = np.random.uniform(size=(K, X.shape[1]))
        pi = np.ones(K) / K

        log_ls = [self.log_likelihood(X, P, pi)]

        for i in range(n_iterations):
            responsibilities = self.expectation(X=X, parameters=P, priors=pi)
            P, pi = self.maximisation(X, responsibilities=responsibilities)
            log_l = self.log_likelihood(X, parameters=P, pi=pi)
            log_ls.append(log_l)
            if log_ls[-1] - log_ls[-2] <= epsilon:
                break

        return P, pi, responsibilities, log_ls

In [92]:
em = EMForMultivariateBernoulli()
P, pi, responsibilities, log_ls = em(K=3, X=dataset, n_iterations=1000, epsilon=1e-6)

In [93]:
log_ls

[-5935.844823871684, -3527.981748917842, -3247.7824906025, -3316.570956146487]